In [1]:
import os
from dotenv import load_dotenv
load_dotenv()


groq_api_key = os.getenv("API_KEY")

In [2]:
from langchain_groq import ChatGroq

model = ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002B10E3E97D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002B10E3F0390>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [7]:
from langchain_core.messages import HumanMessage,SystemMessage

model.invoke([HumanMessage(content="Hi, My name is Arjun and Iam a Gen AI Developer")])

AIMessage(content="Hi Arjun! It's nice to meet you.\n\nThat's awesome that you're a Gen AI Developer! It's such an exciting field right now. \n\nWhat kind of projects are you working on?  I'm always eager to hear about what people are doing with generative AI. 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 22, 'total_tokens': 91, 'completion_time': 0.125454545, 'prompt_time': 0.002135825, 'queue_time': 0.24263195799999998, 'total_time': 0.12759037}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--3162f025-faf6-4a00-86b2-9ab300af7e3b-0', usage_metadata={'input_tokens': 22, 'output_tokens': 69, 'total_tokens': 91})

In [8]:
from langchain_core.messages import AIMessage

model.invoke([
    HumanMessage(content="Hi, My name is Arjun and Iam a Gen AI Developer"),
    AIMessage(content="Hi Arjun,\n\nIt's nice to meet you! That's awesome that you're a Gen AI Developer. It's a really exciting field right now.\n\nWhat kind of projects are you working on?  I'm always interested to hear about what people are building with AI.\n"),
    HumanMessage(content="Hey Whats my name and what do i do?")
])

AIMessage(content="You are Arjun, and you are a Gen AI Developer!  \n\nIs there anything else you'd like to know or talk about? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 104, 'total_tokens': 137, 'completion_time': 0.06, 'prompt_time': 0.005529318, 'queue_time': 0.245730632, 'total_time': 0.065529318}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--5aea5b11-ed5d-4470-896b-3fe0a9e2b39b-0', usage_metadata={'input_tokens': 104, 'output_tokens': 33, 'total_tokens': 137})

In [11]:
## Message History
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store ={}

def get_session_history(session_id:str)-> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model,get_session_history)


In [10]:
config={"configurable":{"session_id":"chat1"}}

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi, My name is Arjun and Iam a Gen AI Developer")],
    config=config
)

In [14]:
response.content

"Hi Arjun,\n\nIt's great to meet you! Being a Gen AI developer is fascinating. What kind of projects are you working on these days? I'm always eager to learn about the innovative things people are building with this technology.  \n\nPerhaps you could tell me about:\n\n* **The specific type of Gen AI you focus on:** (e.g., text generation, image creation, code generation)\n* **A project you're particularly proud of:**\n* **Any challenges you're facing in your work:**\n\nI'm here to listen and learn!  😊  \n\n"

In [15]:
with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config
)

AIMessage(content='Your name is Arjun, as you told me at the beginning of our conversation.  😊 \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 234, 'total_tokens': 256, 'completion_time': 0.04, 'prompt_time': 0.00913723, 'queue_time': 0.243186939, 'total_time': 0.04913723}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--3a6d65c1-09b8-4b6c-8c8e-37e4799cab96-0', usage_metadata={'input_tokens': 234, 'output_tokens': 22, 'total_tokens': 256})

In [17]:
config1={"configurable":{"session_id":"chat2"}}
response =with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)

response.content


"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to use it! 😊\n"

In [19]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant, Answer all the questions to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain = prompt|model

In [20]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Arjun")]})

AIMessage(content="Hi Arjun, it's nice to meet you! \n\nHow can I help you today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 30, 'total_tokens': 56, 'completion_time': 0.047272727, 'prompt_time': 0.002158084, 'queue_time': 0.243241799, 'total_time': 0.049430811}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--b3a93fcc-a083-4550-af66-84b23d982d7f-0', usage_metadata={'input_tokens': 30, 'output_tokens': 26, 'total_tokens': 56})

In [21]:
with_message_history= RunnableWithMessageHistory(chain,get_session_history)

In [22]:
config={"configurable":{"session_id":"chat3"}}
response= with_message_history.invoke(
    [HumanMessage(content="Hi My name is Arjun")],
    config=config
)
response

AIMessage(content="Hi Arjun, it's nice to meet you!  \n\nI'm happy to help with any questions you have.  What can I do for you today? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 30, 'total_tokens': 69, 'completion_time': 0.070909091, 'prompt_time': 0.002296484, 'queue_time': 0.247670358, 'total_time': 0.073205575}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--deaa27ca-5443-4b37-a53f-de5e1b7b3333-0', usage_metadata={'input_tokens': 30, 'output_tokens': 39, 'total_tokens': 69})

In [23]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
        "system",
        "You are a helpful assistant. Answer all questions to the best of your ability in {language}"
        ),
        MessagesPlaceholder(variable_name="messages"),
       
    ]
)
chain = prompt|model

In [30]:
response = chain.invoke({"messages":[HumanMessage(content="Hi my name is Arjun")],"language":"Kannada"})
response.content

'ನಮಸ್ಕಾರ ಅರ್ಜುನ್! 😊 \n\nನಾನು ನಿಮಗೆ ಸಹಾಯಕತ್ವವನ್ನು ನೀಡಲು ತಯಾರಿದಿದ್ದೇನು.  \nನಿಮಗೆ ಏನು ಬೇಕು? \n\n(Hello Arjun! 😊 I am ready to assist you. What can I do for you?) \n'

In [25]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [29]:
config={"configurable":{"session_id":"chat4"}}
response= with_message_history.invoke(
    {'messages':[HumanMessage(content="Hi im Arjun")],"language":"Kannada"},
    config=config
)
response.content

'ನಮಸ್ತೆ ಅರ್ಜುನ್! 😊  \n\nನನಗೆ ನಿಮ್ಮೊಂದಿಗೆ ಮಾತನಾಡಲು ಖುಷಿ 😊 \n ನೀವು ಏನು ತಿಳಿಯಬೇಕು? \n'

In [28]:
response = with_message_history.invoke(
    {"messages":[HumanMessage(content="Whats my name?")],"language":"Kannada"},
    config=config
)

response.content

'ನೀವು ಅರ್ಜುನ್  ಎಂದು ಹೇಳಿದ್ದೀರಿ 😊  \n'

In [38]:
from langchain_core.messages import SystemMessage,trim_messages

trimmer = trim_messages(
    max_tokens=70,
    strategy="last",
    token_counter =model,
    include_system = True,
    allow_partial= False,
    start_on = "human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

d:\Data science\SimplellmLCEL\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Arjun S\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [39]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"Kannada"
    }
)
response.content

'you like vanilla ice cream!  🍦'